# <font color="red"> MBA em IA e Big Data</font>
## <span style="color:red">Redes Neurais e Deep Learning</span>

 

##  Aumentação de Dados e Transferência de Aprendizado

*Roseli Aparecida Francelin Romero*<br>
*ICMC/USP São Carlos*

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from numpy.random import seed
from tensorflow.random import set_seed

import tensorflow_datasets as tfds

tfds.disable_progress_bar()

# selecionamos 20% para treinamento (~5200 imagens) e 10% para validacao
(train_ds, validation_ds), info = tfds.load(
    "malaria",
    split=["train[0%:20%]", "train[20%:30%]"],
    as_supervised=True, 
    with_info=True
)

2022-11-07 18:25:40.163396: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


NonMatchingChecksumError: Artifact https://ceb.nlm.nih.gov/proj/malaria/cell_images.zip, downloaded to /root/tensorflow_datasets/downloads/ceb.nlm.nih.gov_proj_malaria_cell_imagesCLJ1vGxXDKcJmHnYfoar_K3ipRQWtxIVA-imvIbvBbs.zip.tmp.8abb8ff235af482bbfb5ccf58af97730/downloads.html, has wrong checksum:
* Expected: UrlInfo(size=337.08 MiB, checksum='0a949556b2414159b5100192609805376654c4266d8d187be9b1922fad43c668', filename='cell_images.zip')
* Got: UrlInfo(size=108.80 KiB, checksum='15b633e238ab72ea9393f18ffccdb1028ff433dfc68b8bda02d08b0c7b59a9e0', filename='downloads.html')
To debug, see: https://www.tensorflow.org/datasets/overview#fixing_nonmatchingchecksumerror

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 6))
for i, (image, label) in enumerate(train_ds.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(int(label))
    plt.axis("off")


In [ ]:
print(info)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

# redimensionando exemplos e normalizando entre 0-1 tipo float32
img_size = (128, 128)
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, img_size), y))
train_ds = train_ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, img_size), y))
validation_ds = validation_ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

input_shape = img_size+(3,)

In [ ]:
batch_size = 32

train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)

---

### Data Augmentation: 
#### funciona como uma camada de pré-processamento, gerando transformações aleatórias na imagem de entrada. 

Ao longo das épocas é responsável pela perturbação da entrada.

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomFlip("vertical"),
        layers.experimental.preprocessing.RandomRotation(0.2),
    ]
)

In [ ]:
for images, labels in train_ds.take(1):
    plt.figure(figsize=(6, 6))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(images[2], 0), training=True,
        )
        plt.imshow(augmented_image[0].numpy())
        plt.axis("off")


--- 
### Carregando a CNN "MobileNet V2" para ser usada como "backbone" da solução

<font color="red">Tentativa 1: pesos inicializados aleatoriamente

* `weights=None` para que os pesos sejam aleatórios
* `include_top=False` pois não queremos a última camada (top), especifica da ImageNet


In [ ]:
base_model_random = tf.keras.applications.MobileNetV2(
    weights=None,  
    input_shape=(128, 128, 3),
    include_top=False
)

# Permitir treinamento do modelo carregado
base_model_random.trainable = True

# Com base na MobileNetV2 vamos criar nosso modelo
# definimos o tamanho da entrada
inputs = keras.Input(shape=(128, 128, 3))
# aplicamos a transformacao da imagem de entrada
x = data_augmentation(inputs)

# Depois da "aumentacao de dados", temos o modelo base
x = base_model_random(x, training=True)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.25)(x)
outputs = keras.layers.Dense(1)(x)

model_random = keras.Model(inputs, outputs)

model_random.summary()

#### Compilando e treinando o modelo a partir de pessos aleatórios

In [ ]:
seed(1)
set_seed(2)

model_random.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss="binary_crossentropy",
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 20
rand_hist = model_random.fit(train_ds, epochs=epochs, validation_data=validation_ds)

--- 
### Carregando a CNN "MobileNet V2" com pesos pré-treinados

<font color="red">Tentativa 2: uso de pesos treinados na imagenet


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    weights="imagenet",
    input_shape=(128, 128, 3),
    include_top=False
)

# Tornamos o modelo base não treinável, "congelando" os parâmetros
base_model.trainable = False

# Nosso modelo como anteriormente
inputs = keras.Input(shape=(128, 128, 3))
x = data_augmentation(inputs) 

# Incluindo a MobileNetV2 com parametros pré-treinados, mas ainda não treinável
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.25)(x)
outputs = keras.layers.Dense(1)(x) # essa será a única camada treinável
model = keras.Model(inputs, outputs)

model.summary()


#### <font color="red"> Passo 1 (Transfer-Learning): treinar apenas a camada de saída (softmax)


In [ ]:
seed(1)
set_seed(2)

model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss="binary_crossentropy",
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 12
pt_hist1 = model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

#### <font color="red"> Passo 2: (Fine-tuning) ajuste fino do restante dos parâmetros

Executamos mais algumas épocas para tentar melhorar as outras camadas

In [ ]:
# des-congelando o modelo base, permitindo ser treinável
base_model.trainable = True

# note a diferença no summary
model.summary()

# aqui definimos um passo/learning rate bem pequeno para o ajuste-fino
model.compile(
    optimizer=keras.optimizers.Adam(0.00001), 
    loss="binary_crossentropy",
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 8
pt_hist2 = model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

Comparando o histórico das duas abordagens

In [ ]:
pt_tra_hist = np.concatenate((pt_hist1.history['binary_accuracy'], pt_hist2.history['binary_accuracy']), axis=None)
pt_val_hist = np.concatenate((pt_hist1.history['val_binary_accuracy'], pt_hist2.history['val_binary_accuracy']), axis=None)

plt.plot(rand_hist.history['binary_accuracy'])
plt.plot(rand_hist.history['val_binary_accuracy'])
plt.plot(pt_tra_hist)
plt.plot(pt_val_hist)
plt.legend(['rnd_acc','rnd_val_acc','pt_acc','pt_val_acc'])

--- 
### <font color="red"> Obtendo features a partir da CNN "MobileNet V2" com pesos pré-treinados

Esse processo não necessita de treinamento da rede

Vamos obter a saída dessa rede, sem incluir a camada de predição (softmax).
Para obter features, incluímos uma nova camada, de Global Pooling e definimos essa camada como sendo a de saída do modelo.

Aqui também não incluimos o "topo" da rede, cujas camadas dependem de uma entrada de tamanho igual a da ImageNet (224x224) e saída com 1000 classes.


In [ ]:
base_model_extraction = keras.applications.MobileNetV2(
    weights="imagenet",
    input_shape=(128, 128, 3),
    include_top=False
)

# obtemos a camada de saída do modelo carregado (última camada pois não incluímos o topo da rede)
base_output = base_model_extraction.layers[-1].output

# para evitar uma dimensionalidade alta, definimos uma nova camada de saída
feat_layer = keras.layers.GlobalAveragePooling2D()(base_output)

# montamos um novo modelo com a entrada do pré-treinado, e saída criada acima
model3 = keras.models.Model(base_model_extraction.inputs, feat_layer)
# descomente para ver como ficou o modelo final
# model3.summary()

# passando os exemplos de treinamento pela rede sem treinamento
# o "predict" vai nos dar a saída programada, obtida da GlobalAveragePooling2D
features_train = np.array(model3.predict(train_ds))
print('\tDados x features obtidas treinamento: ', features_train.shape)


As features obtidas podem ser utilizadas para treinar classificadores não profundos ou como índices de sistemas de recuperação baseada em conteúdo
